In [29]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns #for plotting
import warnings
from sklearn.cluster import KMeans
import warnings
from sklearn.ensemble import RandomForestClassifier #for the model
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz #plot tree
from sklearn.metrics import roc_curve, auc #for model evaluation
from sklearn.metrics import classification_report #for model evaluation
from sklearn.metrics import confusion_matrix #for model evaluation
from sklearn.model_selection import train_test_split #for data splitting
from sklearn.feature_selection import RFE
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn import svm
import statsmodels.api as sm
#import eli5 #for purmutation importance
#from eli5.sklearn import PermutationImportance
#import shap #for SHAP values
#from pdpbox import pdp, info_plots #for partial plots
np.random.seed(123) #ensure reproducibility
pd.options.mode.chained_assignment = None  #hide any pandas warnings
import scipy.stats as stats
import pylab
%matplotlib inline
pd.pandas.set_option('display.max_columns',None)
from statsmodels.tools.sm_exceptions import ConvergenceWarning
warnings.simplefilter('ignore', ConvergenceWarning)


In [30]:
dt = pd.read_csv("heart.csv")

In [31]:
dt.head(10)

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1
5,57,1,0,140,192,0,1,148,0,0.4,1,0,1,1
6,56,0,1,140,294,0,0,153,0,1.3,1,0,2,1
7,44,1,1,120,263,0,1,173,0,0.0,2,0,3,1
8,52,1,2,172,199,1,1,162,0,0.5,2,0,3,1
9,57,1,2,150,168,0,1,174,0,1.6,2,0,2,1


In [10]:
dt.tail()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
298,57,0,0,140,241,0,1,123,1,0.2,1,0,3,0
299,45,1,3,110,264,0,1,132,0,1.2,1,0,3,0
300,68,1,0,144,193,1,1,141,0,3.4,1,2,3,0
301,57,1,0,130,131,0,1,115,1,1.2,1,1,3,0
302,57,0,1,130,236,0,0,174,0,0.0,1,1,2,0


In [11]:
dt.columns

Index(['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach',
       'exang', 'oldpeak', 'slope', 'ca', 'thal', 'target'],
      dtype='object')

In [32]:
dt.columns = ['age', 'gender', 'chest_pain_type', 'resting_blood_pressure', 'cholesterol', 'fasting_blood_sugar', 'rest_ecg', 'max_heart_rate_achieved',
       'exercise_induced_angina', 'st_depression', 'st_slope', 'num_major_vessels', 'thalassemia', 'target']

In [33]:
dt['gender'][dt['gender'] == 0] = 'female'
dt['gender'][dt['gender'] == 1] = 'male'

dt['chest_pain_type'][dt['chest_pain_type'] == 1] = 'typical angina'
dt['chest_pain_type'][dt['chest_pain_type'] == 2] = 'atypical angina'
dt['chest_pain_type'][dt['chest_pain_type'] == 3] = 'non-anginal pain'
dt['chest_pain_type'][dt['chest_pain_type'] == 4] = 'asymptomatic'

dt['fasting_blood_sugar'][dt['fasting_blood_sugar'] == 0] = 'lower than 120mg/ml'
dt['fasting_blood_sugar'][dt['fasting_blood_sugar'] == 1] = 'greater than 120mg/ml'

dt['rest_ecg'][dt['rest_ecg'] == 0] = 'normal'
dt['rest_ecg'][dt['rest_ecg'] == 1] = 'ST-T wave abnormality'
dt['rest_ecg'][dt['rest_ecg'] == 2] = 'left ventricular hypertrophy'

dt['exercise_induced_angina'][dt['exercise_induced_angina'] == 0] = 'no'
dt['exercise_induced_angina'][dt['exercise_induced_angina'] == 1] = 'yes'

dt['st_slope'][dt['st_slope'] == 1] = 'upsloping'
dt['st_slope'][dt['st_slope'] == 2] = 'flat'
dt['st_slope'][dt['st_slope'] == 3] = 'downsloping'

dt['thalassemia'][dt['thalassemia'] == 1] = 'normal'
dt['thalassemia'][dt['thalassemia'] == 2] = 'fixed defect'
dt['thalassemia'][dt['thalassemia'] == 3] = 'reversable defect'

In [14]:
dt.dtypes

age                          int64
gender                      object
chest_pain_type             object
resting_blood_pressure       int64
cholesterol                  int64
fasting_blood_sugar         object
rest_ecg                    object
max_heart_rate_achieved      int64
exercise_induced_angina     object
st_depression              float64
st_slope                    object
num_major_vessels            int64
thalassemia                 object
target                       int64
dtype: object

In [15]:
dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   age                      303 non-null    int64  
 1   gender                   303 non-null    object 
 2   chest_pain_type          303 non-null    object 
 3   resting_blood_pressure   303 non-null    int64  
 4   cholesterol              303 non-null    int64  
 5   fasting_blood_sugar      303 non-null    object 
 6   rest_ecg                 303 non-null    object 
 7   max_heart_rate_achieved  303 non-null    int64  
 8   exercise_induced_angina  303 non-null    object 
 9   st_depression            303 non-null    float64
 10  st_slope                 303 non-null    object 
 11  num_major_vessels        303 non-null    int64  
 12  thalassemia              303 non-null    object 
 13  target                   303 non-null    int64  
dtypes: float64(1), int64(6), o

# DATA CLEANSING

In [ ]:
dt.isnull().sum()

### Droping Null values because Null values are very less as compared to data size

In [ ]:
dt=dt.dropna()

# DATA INSIGHTS

In [18]:
dt.age.unique()

array([63, 37, 41, 56, 57, 44, 52, 54, 48, 49, 64, 58, 50, 66, 43, 69, 59,
       42, 61, 40, 71, 51, 65, 53, 46, 45, 39, 47, 62, 34, 35, 29, 55, 60,
       67, 68, 74, 76, 70, 38, 77], dtype=int64)

In [ ]:
dt.age.value_counts()

In [ ]:
dt.gender.unique()

In [ ]:
dt.gender.value_counts()

In [ ]:
dt.chest_pain_type.unique()

In [ ]:
dt.chest_pain_type.value_counts()

In [ ]:
dt.resting_blood_pressure.unique()

In [ ]:
dt.resting_blood_pressure.value_counts()

In [ ]:
dt.cholesterol.unique()

In [ ]:
dt.cholesterol.value_counts()

In [ ]:
dt.fasting_blood_sugar.unique()

In [ ]:
dt.fasting_blood_sugar.value_counts()

In [ ]:
dt.rest_ecg.unique()

In [ ]:
dt.rest_ecg.value_counts()

In [ ]:
dt.max_heart_rate_achieved.value_counts()

In [ ]:
dt.max_heart_rate_achieved.unique()

In [ ]:
dt.exercise_induced_angina.unique()

In [ ]:
dt.exercise_induced_angina.value_counts()

In [ ]:
dt.st_depression.value_counts()

In [ ]:
dt.st_depression.value_counts()

In [ ]:
dt.st_slope.unique()

In [ ]:
dt.st_slope.value_counts()

In [ ]:
dt.num_major_vessels.value_counts()

In [ ]:
dt.num_major_vessels.unique()

In [ ]:
dt.thalassemia.unique()

In [ ]:
dt.thalassemia.value_counts()

In [ ]:
from dataprep.datasets import load_dataset
from dataprep.eda import create_report
#df = load_dataset("titanic")
create_report(dt)

# CHECK FOR OUTLIERS

In [ ]:
plt.boxplot(dt.age)

In [ ]:
plt.boxplot(dt.resting_blood_pressure)

In [ ]:
plt.boxplot(dt.cholesterol)

In [ ]:
plt.boxplot(dt.max_heart_rate_achieved)

In [ ]:
plt.boxplot(dt.num_major_vessels)

# STATISTICAL EDA

In [ ]:
dt.mean()

In [ ]:
dt.median()

In [ ]:
dt.mode()

In [ ]:
dt.describe()

In [ ]:
dt.skew()

In [ ]:
dt.corr()

In [ ]:
dataplot = sns.heatmap(dt.corr(),cmap = "PiYG",annot = True)
plt.rcParams['figure.figsize'] = [10,10]

## Heart Diseases Ratio in Dataset
Blue Graph indicate no heart desease and Orange Graph show Heart desease

In [ ]:
def plotTarget():
    sns.countplot(x='target', data=dt, ax=ax)
    for i, p in enumerate(ax.patches):
        count=dt['target'].value_counts().values[i]
        x=p.get_x()+ p.get_width() /2.
        y=p.get_height() + 3
        label='{:1.2f}'.format(count / float(dt.shape[0]))
        ax.text(x, y,label, ha='center')
        
fig_target,ax=plt.subplots(nrows=1, ncols=1, figsize=(5, 2))
plotTarget()

## Select Age as most dependent data on label
Disease Probability Bar Plot

In [ ]:
def plotAge():
    facet_grid = sns.FacetGrid(dt, hue='target')
    facet_grid.map(sns.kdeplot, "age", shade=True, ax=axes[0])
    legend_labels = ['disease false', 'disease true']
    #for t, l in zip(axes[0].get_legend().texts, legend_labels):
        #t.set_text(l)
        
    axes[0].set(xlabel='age', ylabel='density')

    avg = dt[["age", "target"]].groupby(['age'], as_index=False).mean()
    sns.barplot(x='age', y='target', data=avg, ax=axes[1])
    axes[1].set(xlabel='age', ylabel='disease probability')

    plt.clf()

In [ ]:
fig_age, axes = plt.subplots(nrows=2, ncols=1, figsize=(15, 8))

plotAge()

## Ploting Function For Categorical Data " Chest Pain" && "Thalassemia"
Ploting Function For Continoius Data

In [ ]:
category=[('gender',['female','male']),('chest_pain_type',['typical angina','atypical angina','non-anginal pain','asymptomatic']),('fasting_blood_sugar',['lower than 120mg/ml','greater than 120mg/ml']),('rest_ecg',['normal','ST-T wave abnormality','left ventricular hypertrophy']),('exercise_induced_angina',['no','yes']),('st_slope',['upsloping','flat','downsloping']),('thalassemia',['fixed','normal','reversable'])]
continuous = [('resting_blood_pressure'),('cholesterol'),('max_heart_rate_achieved'),('st_depression'),('num_major_vessels')]

        
def plotCategorial(attribute, labels, ax_index):
    sns.countplot(x=attribute, data=dt, ax=axes[ax_index][0])
    sns.countplot(x='target', hue=attribute, data=dt, ax=axes[ax_index][1])
    avg = dt[[attribute, 'target']].groupby([attribute], as_index=False).mean()
    sns.barplot(x=attribute, y='target', hue=attribute, data=avg, ax=axes[ax_index][2])
    
    for t, l in zip(axes[ax_index][1].get_legend().texts, labels):
        t.set_text(l)
    for t, l in zip(axes[ax_index][2].get_legend().texts, labels):
        t.set_text(l)


def plotContinuous(attribute, xlabel, ax_index):
    sns.distplot(dt[[attribute]], ax=axes[ax_index][0])
    axes[ax_index][0].set(xlabel=xlabel, ylabel='density')
    sns.violinplot(x='target', y=attribute, data=dt, ax=axes[ax_index][1])
    
    
def plotGrid(isCategorial):
    if isCategorial:
        [plotCategorial(x[0], x[1], i) for i, x in enumerate(category)] 
    else:
        [plotContinuous(x[0], x[1], i) for i, x in enumerate(continuous)]

In [ ]:
fig_categorial,axes=plt.subplots(nrows=len(category), ncols=3, figsize=(14, 18))
plotGrid(isCategorial=True)

In [ ]:
sns.kdeplot(dt.age)

In [ ]:
stats.probplot(dt.age,plot=pylab)

In [ ]:
sns.kdeplot(dt.max_heart_rate_achieved)

In [ ]:
stats.probplot(dt.max_heart_rate_achieved,plot=pylab)

In [ ]:
def normality(data,feature):
    mlp.figure(figsize=(10,5))
    mlp.subplot(1,2,1)
    sns.kdeplot(data[feature])
    mlp.subplot(1,2,2)
    stats.probplot(data[feature],plot=pylab)
    mlp.show()

In [ ]:
normality(dt,'age')

In [ ]:
normality(dt,'resting_blood_pressure')

In [ ]:
normality(dt,'cholesterol')

In [ ]:
normality(dt,'resting_blood_pressure')

In [ ]:
normality(dt,'max_heart_rate_achieved')

In [ ]:
normality(dt,'st_depression')

In [ ]:
normality(dt,'num_major_vessels')

In [ ]:
sns.pairplot(dt,hue="target",size = 3)

# TRANSFORMATIONS(log, sqrt, reciprocal, boxcox)

In [23]:
dt1 = dt.copy(deep = True)

In [24]:
dt1['age_log'] = np.log(dt1['age'])
normality(dt1,'age_log')

NameError: name 'normality' is not defined

In [25]:
dt1['age_rcp'] = 1/dt1.age
normality(dt1,'age_rcp')

NameError: name 'normality' is not defined

In [26]:
dt1['age_sqrt'] = np.sqrt(dt1.age)
normality(dt1,'age_sqrt')

NameError: name 'normality' is not defined

In [27]:
dt1['age_bcx'],parameters = stats.boxcox(dt1['age'])
normality(dt1,'age_bcx')

NameError: name 'normality' is not defined

In [ ]:
dt1['resting_blood_pressure_log'] = np.log(dt1['resting_blood_pressure'])
normality(dt1,'resting_blood_pressure_log')

In [ ]:
dt1['resting_blood_pressure_rcp'] = 1/dt1.resting_blood_pressure
normality(dt1,'resting_blood_pressure_rcp')

In [ ]:
dt1['resting_blood_pressure_sqrt'] = np.sqrt(dt1.resting_blood_pressure)
normality(dt1,'resting_blood_pressure_sqrt')

In [ ]:
dt1['resting_blood_pressure_exp'] = dt1.resting_blood_pressure**(1/1.2)
normality(dt1,'resting_blood_pressure_exp')

In [ ]:
dt1['resting_blood_pressure_box'],parameters = stats.boxcox(dt1['resting_blood_pressure'])
normality(dt1,'resting_blood_pressure_box')

# FEATURE ENGINEERING

In [34]:
X = dt.iloc[:,:-1]
Y = dt['target']

In [35]:
X.head()

,age,gender,chest_pain_type,resting_blood_pressure,cholesterol,fasting_blood_sugar,rest_ecg,max_heart_rate_achieved,exercise_induced_angina,st_depression,st_slope,num_major_vessels,thalassemia
0,63,male,non-anginal pain,145,233,greater than 120mg/ml,normal,150,no,2.3,0,0,normal
1,37,male,atypical angina,130,250,lower than 120mg/ml,ST-T wave abnormality,187,no,3.5,0,0,fixed defect
2,41,female,typical angina,130,204,lower than 120mg/ml,normal,172,no,1.4,flat,0,fixed defect
3,56,male,typical angina,120,236,lower than 120mg/ml,ST-T wave abnormality,178,no,0.8,flat,0,fixed defect
4,57,female,0,120,354,lower than 120mg/ml,ST-T wave abnormality,163,yes,0.6,flat,0,fixed defect


In [36]:
Y.head()

0    1
1    1
2    1
3    1
4    1
Name: target, dtype: int64

# ONE HOT ENCODER

In [37]:
x = pd.get_dummies(X, drop_first=True)

In [38]:
x.head()

,age,resting_blood_pressure,cholesterol,max_heart_rate_achieved,st_depression,num_major_vessels,gender_male,chest_pain_type_atypical angina,chest_pain_type_non-anginal pain,chest_pain_type_typical angina,fasting_blood_sugar_lower than 120mg/ml,rest_ecg_left ventricular hypertrophy,rest_ecg_normal,exercise_induced_angina_yes,st_slope_flat,st_slope_upsloping,thalassemia_fixed defect,thalassemia_normal,thalassemia_reversable defect
0,63,145,233,150,2.3,0,1,0,1,0,0,0,1,0,0,0,0,1,0
1,37,130,250,187,3.5,0,1,1,0,0,1,0,0,0,0,0,1,0,0
2,41,130,204,172,1.4,0,0,0,0,1,1,0,1,0,1,0,1,0,0
3,56,120,236,178,0.8,0,1,0,0,1,1,0,0,0,1,0,1,0,0
4,57,120,354,163,0.6,0,0,0,0,0,1,0,0,1,1,0,1,0,0


In [39]:
x.shape

(303, 19)

# FEATURE SELECTION

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

In [ ]:
### Apply SelectKBest Algorithm
ordered_rank_features=SelectKBest(score_func=chi2,k=19)
ordered_feature=ordered_rank_features.fit(x,Y)

In [ ]:
dtscores=pd.DataFrame(ordered_feature.scores_,columns=["Score"])
dtcolumns=pd.DataFrame(x.columns)

In [ ]:
features_rank=pd.concat([dtcolumns,dtscores],axis=1)

In [ ]:
features_rank.columns=['Features','Score']
features_rank

In [ ]:
features_rank.nlargest(10,'Score')

# Feature Importance




This technique gives you a score for each feature of your data,the higher the score mor relevant it is

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
#import matplotlib.pyplot as plt
model=ExtraTreesClassifier()
model.fit(x,Y)

In [ ]:
print(model.feature_importances_)

In [ ]:
ranked_features=pd.Series(model.feature_importances_,index=x.columns)
ranked_features.nlargest(10).plot(kind='barh')
plt.show()

# Remove the uncorrelated features

In [ ]:
threshold=0.2

In [ ]:
# find and remove correlated features
def correlation(dataset, threshold):
    col_corr = set()  # Set of all the names of correlated columns
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold: # we are interested in absolute coeff value
                colname = corr_matrix.columns[i]  # getting the name of column
                col_corr.add(colname)
    return col_corr

In [ ]:
correlation(x,threshold)

# INFORMATION GAIN

In [ ]:
from sklearn.feature_selection import mutual_info_classif

In [ ]:
mutual_info=mutual_info_classif(x,Y)

In [ ]:
mutual_data=pd.Series(mutual_info,index=x.columns)
mutual_data.sort_values(ascending=False)

In [ ]:
selected_features = []
rfe = RFE(LogisticRegression())

rfe.fit(x.values, Y.values)

for i, feature in enumerate(x.columns.values):
    if rfe.support_[i]:
        selected_features.append(feature)

selected_X = x[selected_features]
selected_y = Y

lm = sm.Logit(selected_y, selected_X)
result = lm.fit()

print(result.summary2())

# DATA SPLITING

In [40]:
X_train, X_test, Y_train, Y_test = train_test_split(x,Y, test_size = .2, random_state=10) #split the data

In [41]:
X_train

,age,resting_blood_pressure,cholesterol,max_heart_rate_achieved,st_depression,num_major_vessels,gender_male,chest_pain_type_atypical angina,chest_pain_type_non-anginal pain,chest_pain_type_typical angina,fasting_blood_sugar_lower than 120mg/ml,rest_ecg_left ventricular hypertrophy,rest_ecg_normal,exercise_induced_angina_yes,st_slope_flat,st_slope_upsloping,thalassemia_fixed defect,thalassemia_normal,thalassemia_reversable defect
286,59,134,204,162,0.8,2,1,0,1,0,1,0,0,0,1,0,1,0,0
102,63,140,195,179,0.0,2,0,0,0,1,1,0,0,0,1,0,1,0,0
242,64,145,212,132,2.0,2,1,0,0,0,1,0,1,0,0,1,0,1,0
65,35,138,183,182,1.4,0,0,0,0,0,1,0,0,0,1,0,1,0,0
35,46,142,177,160,1.4,0,0,1,0,0,1,0,1,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156,47,130,253,179,0.0,0,1,1,0,0,1,0,0,0,1,0,1,0,0
123,54,108,267,167,0.0,0,0,1,0,0,1,0,1,0,1,0,1,0,0
15,50,120,219,158,1.6,0,0,1,0,0,1,0,0,0,0,1,1,0,0
125,34,118,210,192,0.7,0,0,0,0,1,1,0,0,0,1,0,1,0,0


In [ ]:
X_test

# LAZY PREDICT TO DECIDE BEST ALGORITHMS

In [ ]:
import lazypredict
from lazypredict.Supervised import LazyClassifier

In [ ]:
# Defines and builds the lazyclassifier
clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models_train,predictions_train = clf.fit(X_train, X_train, y_train, y_train)
models_test,predictions_test = clf.fit(X_train, X_test, y_train, y_test)

# Prints the model performance
models_train

# DECISION TREE CLASSIFIER

In [43]:
dt_clf_gini = DecisionTreeClassifier(criterion = "gini",
                                     random_state = 100,
                                     max_depth = 5,
                                     min_samples_leaf = 5)
 
dt_clf_gini.fit(X_train, Y_train)
y_pred_gini = dt_clf_gini.predict(X_test)
 
print ("Decision Tree using Gini Index\nAccuracy is ",accuracy_score(Y_test, y_pred_gini)*100 )

Decision Tree using Gini Index
Accuracy is  75.40983606557377


In [ ]:
model = RandomForestClassifier(max_depth=5)
model.fit(X_train, y_train)

In [ ]:
estimator = model.estimators_[1]
feature_names = [i for i in X_train.columns]

y_train_str = y_train.astype('str')
y_train_str[y_train_str == '0'] = 'no disease'
y_train_str[y_train_str == '1'] = 'disease'
y_train_str = y_train_str.values

In [ ]:
#code from https://towardsdatascience.com/how-to-visualize-a-decision-tree-from-a-random-forest-in-python-using-scikit-learn-38ad2d75f21c

export_graphviz(estimator, out_file='tree.dot', feature_names = feature_names,
                class_names = y_train_str,
                rounded = True, proportion = True, 
                label='root',
                precision = 2, filled = True)

from subprocess import call
call(['dot', '-Tpng', 'tree.dot', '-o', 'tree.png', '-Gdpi=100'])

from IPython.display import Image
Image(filename = 'tree.png')

In [44]:
y_predict = model.predict(X_test)
y_pred_quant = model.predict_proba(X_test)[:, 1]
y_pred_bin = model.predict(X_test)

NameError: name 'model' is not defined

In [46]:
random=RandomForestClassifier(criterion='gini',max_depth=5,random_state=100,bootstrap=True,oob_score=False)
random.fit(X_train,Y_train)
pred_random=random.predict(X_test)
print("RandomForest using Gini Index\nAccuracy is ",accuracy_score(Y_test, pred_random)*100)

RandomForest using Gini Index
Accuracy is  80.32786885245902


In [47]:
import pickle
with open('heart_disease.pkl','wb')as file:pickle.dump(pred_random,file)

# LOGISTIC REGRESSION 

In [ ]:
lr = LogisticRegression()
lr.fit(X_train,Y_train)

print(f"Accuracy: {lr.score(X_test,Y_test):0.3f}")

# NAIVE BAYES

# K-NN

# STOCHASTIC GRADIENT DESCENT

# SUPPORT VECTOR MACHINE

# PERFORMANCE ANALYSIS

In [ ]:
confusion_matrix = confusion_matrix(y_test, y_pred_bin)
confusion_matrix

In [ ]:
total=sum(sum(confusion_matrix))

sensitivity = confusion_matrix[0,0]/(confusion_matrix[0,0]+confusion_matrix[1,0])
print('Sensitivity : ', sensitivity )

specificity = confusion_matrix[1,1]/(confusion_matrix[1,1]+confusion_matrix[0,1])
print('Specificity : ', specificity)

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, y_pred_quant)

fig, ax = plt.subplots()
ax.plot(fpr, tpr)
ax.plot([0, 1], [0, 1], transform=ax.transAxes, ls="--", c=".3")
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.rcParams['font.size'] = 12
plt.title('ROC curve for heart attack classifier')
plt.xlabel('False Positive Rate (1 - Specificity)')
plt.ylabel('True Positive Rate (Sensitivity)')
plt.grid(True)

In [ ]:
auc(fpr, tpr)

In [ ]:
perm = PermutationImportance(model, random_state=1).fit(X_test, Y_test)
eli5.show_weights(perm, feature_names = X_test.columns.tolist())

In [ ]:
base_features = x.columns.values.tolist()
#base_features.remove('target')

feat_name = 'num_major_vessels'
pdp_dist = pdp.pdp_isolate(model=model, dataset=X_test, model_features=base_features, feature=feat_name)

pdp.pdp_plot(pdp_dist, feat_name)
plt.show()

In [ ]:
feat_name = 'age'
pdp_dist = pdp.pdp_isolate(model=model, dataset=X_test, model_features=base_features, feature=feat_name)

pdp.pdp_plot(pdp_dist, feat_name)
plt.show()

In [ ]:
feat_name = 'st_depression'
pdp_dist = pdp.pdp_isolate(model=model, dataset=X_test, model_features=base_features, feature=feat_name)

pdp.pdp_plot(pdp_dist, feat_name)
plt.show()

In [ ]:
inter1  =  pdp.pdp_interact(model=model, dataset=X_test, model_features=base_features, features=['st_slope_upsloping', 'st_depression'])

pdp.pdp_interact_plot(pdp_interact_out=inter1, feature_names=['st_slope_upsloping', 'st_depression'], plot_type='contour')
plt.show()

inter1  =  pdp.pdp_interact(model=model, dataset=X_test, model_features=base_features, features=['st_slope_flat', 'st_depression'])

pdp.pdp_interact_plot(pdp_interact_out=inter1, feature_names=['st_slope_flat', 'st_depression'], plot_type='contour')
plt.show()

In [ ]:
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_test)

shap.summary_plot(shap_values[1], X_test, plot_type="bar")

In [ ]:
shap.summary_plot(shap_values[1], X_test)

In [ ]:
def heart_disease_risk_factors(model, patient):

    explainer = shap.TreeExplainer(model)
    shap_values = explainer.shap_values(patient)
    shap.initjs()
    return shap.force_plot(explainer.expected_value[1], shap_values[1], patient)

In [ ]:
data_for_prediction = X_test.iloc[1,:].astype(float)
heart_disease_risk_factors(model, data_for_prediction)

In [ ]:
data_for_prediction = X_test.iloc[3,:].astype(float)
heart_disease_risk_factors(model, data_for_prediction)

In [ ]:
ax2 = fig.add_subplot(224)
shap.dependence_plot('num_major_vessels', shap_values[1], X_test, interaction_index="st_depression")

In [ ]:
shap_values = explainer.shap_values(X_train.iloc[:50])
shap.force_plot(explainer.expected_value[1], shap_values[1], X_test.iloc[:50])